In [1]:
import urllib.request
import bs4 as bs
import re
import nltk
import string
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /home/annie/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/annie/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
# Function to perform lemmatization
def perform_lemmatization(tokens):
    wnlemmatizer = nltk.stem.WordNetLemmatizer()
    return [wnlemmatizer.lemmatize(token) for token in tokens]


In [3]:
# Function to process text by removing punctuation and lemmatizing
def get_processed_text(document):
    punctuation_removal = dict((ord(punctuation), None) for punctuation in string.punctuation)
    return perform_lemmatization(nltk.word_tokenize(document.lower().translate(punctuation_removal)))



In [4]:
# Greeting responses
greeting_inputs = ("hey", "good morning", "good evening", "morning", "evening", "hi", "whatsup","hello")
greeting_responses = ["hey", "hey hows you?", "*nods*", "hello, how you doing", "hello", "Welcome, I am good and you"]


In [5]:

# Function to generate greeting response
def generate_greeting_response(greeting):
    for token in greeting.split():
        if token.lower() in greeting_inputs:
            return random.choice(greeting_responses)

In [6]:
# Function to generate response based on user input
def generate_response(user_input):
    bot_response = ''
    data_sentences.append(user_input)

    word_vectorizer = TfidfVectorizer(tokenizer=get_processed_text, stop_words='english')
    all_word_vectors = word_vectorizer.fit_transform(data_sentences)
    similar_vector_values = cosine_similarity(all_word_vectors[-1], all_word_vectors)
    similar_sentence_number = similar_vector_values.argsort()[0][-2]

    matched_vector = similar_vector_values.flatten()
    matched_vector.sort()
    vector_matched = matched_vector[-2]

    if vector_matched == 0:
        bot_response = bot_response + "I am sorry, I could not understand you"
        return bot_response
    else:
        bot_response = bot_response + data_sentences[similar_sentence_number]
        return bot_response


In [ ]:
# Initial message to the user
beginning_message = "Hello, I am a WiKi Bot. You can ask me any question regarding a specific Wikipedia link, drop a link and we can get started right away!!"
print(beginning_message)

# Prompt user for a Wikipedia URL
wiki_url = input("Please enter a Wikipedia URL: ")

# Fetch and process data from the URL
get_link = urllib.request.urlopen(wiki_url)
get_link = get_link.read()
data = bs.BeautifulSoup(get_link, 'lxml')
data_paragraphs = data.find_all('p')
data_text = ''

for para in data_paragraphs:
    data_text += para.text

data_text = data_text.lower()
data_text = re.sub(r'\[[0-9]*\]', ' ', data_text)
data_text = re.sub(r'\s+', ' ', data_text)
data_sentences = nltk.sent_tokenize(data_text)
data_words = nltk.word_tokenize(data_text)

# Dialogue loop
continue_dialogue = True
while continue_dialogue:
    human_text = input().lower()
    if human_text != 'bye' or human_text != 'byee' or human_text != 'Goodbye' or human_text != 'goodbye':
        if human_text in ['thanks', 'thank you very much', 'thank you']:
            continue_dialogue = False
            print("WiKi Bot: Most welcome")
        else:
            if generate_greeting_response(human_text) is not None:
                print("WiKi Bot: " + generate_greeting_response(human_text))
            else:
                print("WiKi Bot: ", end="")
                print(generate_response(human_text))
                data_sentences.remove(human_text)
    else:
        continue_dialogue = False
        print("WiKi Bot: Good bye and take care of yourself...")


Hello, I am a WiKi Bot. You can ask me any question regarding a specific Wikipedia link, drop a link and we can get started right away!!


Please enter a Wikipedia URL:  https://en.wikipedia.org/wiki/Mars
